In [1]:
import json
import tensorflow as tf
from recommenders.models.sasrec.model import SASREC

# Load config
with open("data/ckpt/sasrec_config.json") as f:
    config = json.load(f)

# Recreate model
model = SASREC(
    item_num=config["item_num"],
    seq_max_len=config["seq_max_len"],
    num_blocks=config["num_blocks"],
    embedding_dim=config["embedding_dim"],
    attention_dim=config["attention_dim"],
    attention_num_heads=config["attention_num_heads"],
    dropout_rate=config["dropout_rate"],
    conv_dims=config["conv_dims"],
    l2_reg=config["l2_reg"],
    num_neg_test=config["num_neg_test"]
)

# Restore weights
ckpt = tf.train.Checkpoint(model=model)
ckpt.restore("data/ckpt/sasrec.ckpt").expect_partial()

2025-05-04 14:23:49.377183: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-04 14:23:49.409295: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-04 14:23:49.409321: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-04 14:23:49.409337: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-04 14:23:49.413900: I tensorflow/core/platform/cpu_feature_g

FileNotFoundError: [Errno 2] No such file or directory: 'data/ckpt/sasrec_config.json'

In [ ]:
import tensorflow as tf
import numpy as np

maxlen = config["seq_max_len"]
item_num = config["item_num"]
num_neg_test = config["num_neg_test"]

user_seq = [
    1, 3, 1, 3, 113, 3, 33, 1, 741, 741, 741, 3, 1, 741, 133, 1, 3, 33, 3, 1,
    1, 1, 1432, 1, 1, 3, 1432, 3, 1, 1, 33, 1, 3, 1, 3, 1, 1432, 1432, 797, 3,
    3, 1, 1
]
input_seq = user_seq[-maxlen:]
input_seq = [0] * (maxlen - len(input_seq)) + input_seq

all_items = set(range(1, item_num + 1))
interacted_items = set(user_seq)
candidates = np.array(list(all_items - interacted_items))

scores = []
for item in candidates:
    # Prepare input: positive + num_neg_test negatives (here, just the candidate as positive, no negatives)
    inputs = {
        "input_seq": np.array([input_seq]),
        "candidate": np.array([[item] + [0]*num_neg_test])  # pad negatives with zeros or random negatives
    }
    score = model.predict(inputs)[0, 0]  # get the score for the positive item
    scores.append(score)

scores = np.array(scores)
N = 10
top_indices = np.argsort(scores)[::-1][:N]
recommended_items = candidates[top_indices]
print("Top-N recommended item IDs:", recommended_items)

Top-N recommended item IDs: [ 15  35 103 868  39 955  71  93 303 122]
